# Machinaal Leren - Sprint 3: Multiple data sources

## Task:
Use the images or external data sources (eg. Ghent open data)

Possible tasks:
* Use the images to find similar listings (e.g. similar interior style)
* Cluster the images
* Automatically detect attributes of the listing (e.g. garden, bath, shower, washing machine, ...) based on the images
* Find duplicate listings based on the images
* Detect anomalies (rooms that look very different)
* Predict which room a picture is taken in (bedroom, bathroom, outside, ...)
* Combine with external data to better predict the price (e.g. location of public transport, proximity to attractions, ...)
* ...

## Work table

|Task|Peter Bonnarens|Philip Kukoba|Lennert Franssens|
|------|------|------|------|
|1. Loading and cleaning dataset  | X |  |  |
|2. Find similar listings  |  |  |   |
|3. Find duplicate listings  |   |  |   |
|4. Cluster the images  |   |   |  |
|5. Detect attributes of the listing  |   |   |  |
|6. Detect anomalies  |   |   |  |
|7. Predict which room a picture is taken in  |   |   |  |
|8. Use external data to better predict the price  |   |   |  |
|9. Extra 1  |   |   |  |
|10. Extra 2  |   |   |  |
|11. Extra 3  |   |   |  |

# **1 - LOADING AND CLEANING THE DATASET**

## Import packages and read dataset
Here we import the needed packages for this project and read the 'listings' dataset.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# remove warnings from output
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# matplotplib figure size in inches
plt.rcParams['figure.figsize'] = 15,12

# reading the dataset and making a pandas dataframe
listings = pd.read_csv("data/listings.csv")

## Cleaning the 'listings' dataset

TODO: write text

In [ ]:
# TODO: clean the listings dataset

# **2 - FINDING SIMILAR LISTINGS BASED ON THE IMAGES**

# **3 - FINDING DUPLICATE LISTINGS BASED ON THE IMAGES**

# **4 - CLUSTERING THE IMAGES**

# **5 - AUTOMATICALLY DETECT ATTRIBUTES OF LISTINGS BASED ON THE IMAGES**

# **6 - DETECTING ANOMALIES BASED ON THE IMAGES**

# **7 - PREDICTING WHICH ROOM A PICTURE IS TAKEN IN**

# **8 - USING EXTERNAL DATA TO BETTER PREDICT THE PRICE**

# **9 - EXTRA 1**

# **10 - EXTRA 2**

# **11 - EXTRA 3**